In [30]:
import pandas as pd
import dtale
import plotly.express as px 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

In [2]:
df = pd.read_csv("../data/SmileData.csv",skiprows=2)

C:\Users\Joselit\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (2,32,36,53,58,59,77,83,89,93,105,127,141,144) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df.head()

,CallReportNum,ReportVersion,LinkedToCallReportNum,CallDateAndTimeStart,CallDateAndTimeEnd,CallLength,CallerNum,PhoneWorkerNum,WasRealCall,WasHangup,...,SMILE ON 60+ Screening - Treatment Urgency,SMILE ON 60+ Screening - Untreated Decay,SMILE ON 60+ Site Information - Enroller ID,SMILE ON 60+ Site Information - Enroller Location,SMILE ON 60+ Treatment Plan - Action: Upload After Picture on Finish Tab,SMILE ON 60+ Treatment Plan - Is the treatment plan completed?,SMILE ON 60+ Treatment Plan - Was a treatment plan developed?,"SMILE ON 60+ Treatment Plan - Was the enrollee able to be restored to function (can chew) and ""social six"" esthetics (top front six teeth are present and disease free)",SMILE ON 60+ Verification of Eligibility - Transportation Arranged,SMILE ON 60+ Verification of Eligibility - Verification of Eligibility
0,93115324,SMILE ON 60+ Registration,NaN,9/30/2021 16:56,9/30/2021 16:57,1,3638169,123747,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,93115188,SMILE ON 60+ Registration,NaN,9/30/2021 16:53,9/30/2021 16:53,0,4431194,123747,NaN,NaN,...,NaN,NaN,NaN,Referral Follow-Up,NaN,NaN,NaN,NaN,NaN,NaN
2,93115081,SMILE ON 60+ Clinical,NaN,9/30/2021 16:50,9/30/2021 16:51,1,2941341,161979,NaN,NaN,...,NaN,NaN,Christ Community Health Services- Broad Avenue,Provider Site,NaN,No,Yes,No,NaN,NaN
3,93115032,SMILE ON 60+ Clinical,NaN,9/30/2021 16:48,9/30/2021 16:49,1,3823933,161979,NaN,NaN,...,NaN,NaN,Christ Community Health Services- Broad Avenue,Provider Site,NaN,No,Yes,No,NaN,NaN
4,93114968,SMILE ON 60+ Clinical,NaN,9/30/2021 16:46,9/30/2021 16:48,2,3348256,161979,NaN,NaN,...,NaN,NaN,Christ Community Health Services- Broad Avenue,Provider Site,NaN,No,Yes,No,NaN,NaN


In [4]:
empty_call_df = ['LinkedToCallReportNum','WasRealCall','WasHangup','WasSexCall','WasWrongNumber','WasPrankCall','WasSilentCall']
#print(empty_call_df)

In [5]:
df.drop(empty_call_df, inplace=True, axis=1)

In [6]:
empty_loc_df = ['VolunteerComments', 'CallersFeedback', 'TextField2','TextField3','TextField4','TextField5','TextField6','TextField7','TextField8','TextField9','TextField10', 'IPAddress', 'VolIdentifier','GeoCode','GeoAssignment','CensusDivision','CensusTrack','CensusReportingArea','211Region','ThirdPartyName', 'ThirdPartyOrganization', 'ThirdPartyAddress','ThirdPartyCity','ThirdPartyCounty', 'ThirdPartyStateProvince','ThirdPartyPostalCode']
#print(empty_loc_df)

In [7]:
df.drop(empty_loc_df, inplace=True, axis=1)

In [8]:
misc_df = ['PhoneType','Feedback','Reviewed','FeedbackStatus','OrgNum', 'CountryName']

In [9]:
df.drop(misc_df, inplace=True, axis=1)

In [10]:
#df.head()

In [11]:
clean_df = df.dropna(axis=1, how='all')
#clean_df.head()

In [12]:
clean_df.head()

,CallReportNum,ReportVersion,CallDateAndTimeStart,CallDateAndTimeEnd,CallLength,CallerNum,PhoneWorkerNum,CityName,CountyName,StateProvince,...,SMILE ON 60+ Screening - Treatment Urgency,SMILE ON 60+ Screening - Untreated Decay,SMILE ON 60+ Site Information - Enroller ID,SMILE ON 60+ Site Information - Enroller Location,SMILE ON 60+ Treatment Plan - Action: Upload After Picture on Finish Tab,SMILE ON 60+ Treatment Plan - Is the treatment plan completed?,SMILE ON 60+ Treatment Plan - Was a treatment plan developed?,"SMILE ON 60+ Treatment Plan - Was the enrollee able to be restored to function (can chew) and ""social six"" esthetics (top front six teeth are present and disease free)",SMILE ON 60+ Verification of Eligibility - Transportation Arranged,SMILE ON 60+ Verification of Eligibility - Verification of Eligibility
0,93115324,SMILE ON 60+ Registration,9/30/2021 16:56,9/30/2021 16:57,1,3638169,123747,Smyrna,Rutherford,TN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,93115188,SMILE ON 60+ Registration,9/30/2021 16:53,9/30/2021 16:53,0,4431194,123747,Nashville,Davidson,TN,...,NaN,NaN,NaN,Referral Follow-Up,NaN,NaN,NaN,NaN,NaN,NaN
2,93115081,SMILE ON 60+ Clinical,9/30/2021 16:50,9/30/2021 16:51,1,2941341,161979,Memphis,Shelby,TN,...,NaN,NaN,Christ Community Health Services- Broad Avenue,Provider Site,NaN,No,Yes,No,NaN,NaN
3,93115032,SMILE ON 60+ Clinical,9/30/2021 16:48,9/30/2021 16:49,1,3823933,161979,Memphis,Shelby,TN,...,NaN,NaN,Christ Community Health Services- Broad Avenue,Provider Site,NaN,No,Yes,No,NaN,NaN
4,93114968,SMILE ON 60+ Clinical,9/30/2021 16:46,9/30/2021 16:48,2,3348256,161979,Memphis,Shelby,TN,...,NaN,NaN,Christ Community Health Services- Broad Avenue,Provider Site,NaN,No,Yes,No,NaN,NaN


#remove and replace name columns
smileon.columns = [x.replace('current', 'replacement') for x in smileon.columns]

In [13]:
# Set it to None to display all columns in the dataframe
pd.set_option('display.max_columns', None)
clean_df.head()

,CallReportNum,ReportVersion,CallDateAndTimeStart,CallDateAndTimeEnd,CallLength,CallerNum,PhoneWorkerNum,CityName,CountyName,StateProvince,PostalCode,Narrative,EnteredByWorkerNum,EnteredOn,FeedbackFromPhoneWorkerNum,FeedbackFromPhoneWorkerName,ReferralsMade,SMILE ON 60+ Demographic Information - # of People in Household,Birth Year,SMILE ON 60+ Demographic Information - Other Language,SMILE ON 60+ Notes (Optional) - Notes,SMILE ON 60+ Oral Care Encounter - 1351 - Quantity,SMILE ON 60+ Oral Care Encounter - 1354 - Quantity,SMILE ON 60+ Oral Care Encounter - 2140 - Quantity,SMILE ON 60+ Oral Care Encounter - 2150 - Quantity,SMILE ON 60+ Oral Care Encounter - 2160 - Quantity,SMILE ON 60+ Oral Care Encounter - 2161 - Quantity,SMILE ON 60+ Oral Care Encounter - 230 - Quantity,SMILE ON 60+ Oral Care Encounter - 2330 - Quantity,SMILE ON 60+ Oral Care Encounter - 2331 - Quantity,SMILE ON 60+ Oral Care Encounter - 2332 - Quantity,SMILE ON 60+ Oral Care Encounter - 2335 - Quantity,SMILE ON 60+ Oral Care Encounter - 2390 - Quantity,SMILE ON 60+ Oral Care Encounter - 2391 - Quantity,SMILE ON 60+ Oral Care Encounter - 2392 - Quantity,SMILE ON 60+ Oral Care Encounter - 2393 - Quantity,SMILE ON 60+ Oral Care Encounter - 2394 - Quantity,SMILE ON 60+ Oral Care Encounter - 2915 - Quantity,SMILE ON 60+ Oral Care Encounter - 2920 - Quantity,SMILE ON 60+ Oral Care Encounter - 2933 - Quantity,SMILE ON 60+ Oral Care Encounter - 2940 - Quantity,SMILE ON 60+ Oral Care Encounter - 2950 - Quantity,SMILE ON 60+ Oral Care Encounter - 2954 - Quantity,SMILE ON 60+ Oral Care Encounter - 2980 - Quantity,SMILE ON 60+ Oral Care Encounter - 3220 - Quantity,SMILE ON 60+ Oral Care Encounter - 4341 - Quantity,SMILE ON 60+ Oral Care Encounter - 4342 - Quantity,SMILE ON 60+ Oral Care Encounter - 6930 - Quantity,SMILE ON 60+ Oral Care Encounter - 6980 - Quantity,SMILE ON 60+ Oral Care Encounter - 7111 - Quantity,SMILE ON 60+ Oral Care Encounter - 7140 - Quantity,SMILE ON 60+ Oral Care Encounter - 7210 - Quantity,SMILE ON 60+ Oral Care Encounter - 7220 - Quantity,SMILE ON 60+ Oral Care Encounter - 7230 - Quantity,SMILE ON 60+ Oral Care Encounter - 7240 - Quantity,SMILE ON 60+ Oral Care Encounter - 7241 - Quantity,SMILE ON 60+ Oral Care Encounter - 7250 - Quantity,SMILE ON 60+ Oral Care Encounter - 7261 - Quantity,SMILE ON 60+ Oral Care Encounter - 7286 - Quantity,SMILE ON 60+ Oral Care Encounter - 7310 - Quantity,SMILE ON 60+ Oral Care Encounter - 7311 - Quantity,SMILE ON 60+ Oral Care Encounter - 7320 - Quantity,SMILE ON 60+ Oral Care Encounter - 7321 - Quantity,SMILE ON 60+ Oral Care Encounter - 7410 - Quantity,SMILE ON 60+ Oral Care Encounter - 7411 - Quantity,SMILE ON 60+ Oral Care Encounter - 7471 - Quantity,SMILE ON 60+ Oral Care Encounter - 7510 - Quantity,SMILE ON 60+ Oral Care Encounter - 7971 - Quantity,SMILE ON 60+ Oral Care Encounter - 7972 - Quantity,SMILE ON 60+ Oral Care Encounter - 9110 - Quantity,SMILE ON 60+ Oral Care Encounter - 9951 - Quantity,SMILE ON 60+ Oral Care Encounter - Lab Assist narrative,SMILE ON 60+ Screening - Last Screening Date,SMILE ON 60+ Verification of Eligibility - Verification Date,SMILE ON 60+ Base-line Oral and Conditions Questions - Are you limited in what you can eat?,SMILE ON 60+ Base-line Oral and Conditions Questions - Dental Clinic,SMILE ON 60+ Base-line Oral and Conditions Questions - Did the client commit to referral?,SMILE ON 60+ Base-line Oral and Conditions Questions - Did you help in these other areas?,SMILE ON 60+ Base-line Oral and Conditions Questions - Do you need help in these other areas?,SMILE ON 60+ Base-line Oral and Conditions Questions - Have you ever felt that the appearance of your mouth and teeth affected your quality of life?,SMILE ON 60+ Base-line Oral and Conditions Questions - How would you describe the condition of your mouth and teeth?,SMILE ON 60+ Base-line Oral and Conditions Questions - What is the next step for client encounter?,SMILE ON 60+ Base-line Oral and Conditions Questions - What is t

In [14]:
clean_df.columns = clean_df.columns.str.replace(r'SMILE ON 60\+ ', '')
clean_df.columns = clean_df.columns.str.replace('Oral Care Encounter - ', 'oec_')
clean_df.columns = clean_df.columns.str.replace(' - Quantity', '')
clean_df.rename(columns={'Birth Year':'Birth_Year'}, inplace=True)

<ipython-input-14-23d2b75e73b0>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_df.columns = clean_df.columns.str.replace(r'SMILE ON 60\+ ', '')
C:\Users\Joselit\anaconda3\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [67]:
pd.set_option('display.max_columns', None)
clean_df.head(5)

,CallReportNum,ReportVersion,CallDateAndTimeStart,CallDateAndTimeEnd,CallLength,CallerNum,PhoneWorkerNum,CityName,CountyName,StateProvince,PostalCode,Narrative,EnteredByWorkerNum,EnteredOn,FeedbackFromPhoneWorkerNum,FeedbackFromPhoneWorkerName,ReferralsMade,Demographic Information - # of People in Household,Birth_Year,Demographic Information - Other Language,Notes (Optional) - Notes,oec_1351,oec_1354,oec_2140,oec_2150,oec_2160,oec_2161,oec_230,oec_2330,oec_2331,oec_2332,oec_2335,oec_2390,oec_2391,oec_2392,oec_2393,oec_2394,oec_2915,oec_2920,oec_2933,oec_2940,oec_2950,oec_2954,oec_2980,oec_3220,oec_4341,oec_4342,oec_6930,oec_6980,oec_7111,oec_7140,oec_7210,oec_7220,oec_7230,oec_7240,oec_7241,oec_7250,oec_7261,oec_7286,oec_7310,oec_7311,oec_7320,oec_7321,oec_7410,oec_7411,oec_7471,oec_7510,oec_7971,oec_7972,oec_9110,oec_9951,oec_Lab Assist narrative,Screening - Last Screening Date,Verification of Eligibility - Verification Date,Base-line Oral and Conditions Questions - Are you limited in what you can eat?,Base-line Oral and Conditions Questions - Dental Clinic,Base-line Oral and Conditions Questions - Did the client commit to referral?,Base-line Oral and Conditions Questions - Did you help in these other areas?,Base-line Oral and Conditions Questions - Do you need help in these other areas?,Base-line Oral and Conditions Questions - Have you ever felt that the appearance of your mouth and teeth affected your quality of life?,Base-line Oral and Conditions Questions - How would you describe the condition of your mouth and teeth?,Base-line Oral and Conditions Questions - What is the next step for client encounter?,Base-line Oral and Conditions Questions - What is the next step for clients encounter?,Case Management Information - Dental Clinic,Case Management Information - Transportation Barriers,Case Management Information - What barriers have kept you from finding a consistent dental home? (Select all that apply),Case Management Information - What is the outcome for the client?,Clinic Information - Action: Upload Before Picture on Finish Tab,Clinic Information - Is patient of record date if yes last seen?,Clinic Information - Was initial appointment made?,Clinic Information - Was initial transportation assistance or referral provided?,Demographic Information - Gender,Demographic Information - Household Income Level,Demographic Information - How did you first hear about the program?,Demographic Information - Preferred Language/Language Needed,Demographic Information - Race/Ethnicity,Demographic Information - What other assistance programs are you on?,Demographic Information - When is the last time you visited your dentist?,oec_What care was provided to the enrollee? (Select all that apply),Presumptive Eligibility - Dental Insurance?,Presumptive Eligibility - Do they pre-qualify for SMILE ON 60+?,Presumptive Eligibility - What is the next step for clients encounter?,Presumptive Eligibility - What is the next step for clients encounter?,Screening - # of Lower Natural Teeth,Screening - # of Upper Natural Teeth,Screening - Do you have removable dentures?,Screening - Do you usually wear your denture(s) when you eat?,Screening - Experienced any problems related to mouth/teeth during past 12 months?,Screening - How often do you brush your teeth?,Screening - Need for Periodontal Care,Screening - Root Fragments,Screening - Suspicious Soft Tissue Lesions,Screening - Treatment Urgency,Screening - Untreated Decay,Site Information - Enroller ID,Site Information - Enroller Location,Treatment Plan - Action: Upload After Picture on Finish Tab,Treatment Plan - Is the treatment plan completed?,Treatment Plan - Was a treatment plan developed?,"Treatment Plan - Was the enrollee able to be restored to function (can chew) and ""social six"" esthetics (top front six teeth are present and disease free)",Verification of Eligibility - Transportation Arranged,Verification of Eligibility - Verification of Eligibility
0,93115324,SMILE ON 60+ Registration,9/30/2

In [ ]:
# oec_ to be continued if needed
new_names = {'oec_1351':'Sealant-per tooth',
             'oec_1354':'Silver-oin per tooth', 
             'oec_2140':'Amalgam-one surface permanent',
             'oec_2150':'Amalgam-two surfaces permanent',
            'oec_2160':'Amalgam-three surfaces permanent',
            'oec_2161':'Amalgam-four surfaces permanent'}

In [16]:
#clean_df.rename(columns=new_names, inplace=True)
#clean_df.head()

In [25]:
df_sorted =clean_df.sort_values(by=['CallerNum','Clinic Information - Was initial appointment made?'])
df_sorted.head(5)

,CallReportNum,ReportVersion,CallDateAndTimeStart,CallDateAndTimeEnd,CallLength,CallerNum,PhoneWorkerNum,CityName,CountyName,StateProvince,PostalCode,Narrative,EnteredByWorkerNum,EnteredOn,FeedbackFromPhoneWorkerNum,FeedbackFromPhoneWorkerName,ReferralsMade,Demographic Information - # of People in Household,Birth_Year,Demographic Information - Other Language,Notes (Optional) - Notes,oec_1351,oec_1354,oec_2140,oec_2150,oec_2160,oec_2161,oec_230,oec_2330,oec_2331,oec_2332,oec_2335,oec_2390,oec_2391,oec_2392,oec_2393,oec_2394,oec_2915,oec_2920,oec_2933,oec_2940,oec_2950,oec_2954,oec_2980,oec_3220,oec_4341,oec_4342,oec_6930,oec_6980,oec_7111,oec_7140,oec_7210,oec_7220,oec_7230,oec_7240,oec_7241,oec_7250,oec_7261,oec_7286,oec_7310,oec_7311,oec_7320,oec_7321,oec_7410,oec_7411,oec_7471,oec_7510,oec_7971,oec_7972,oec_9110,oec_9951,oec_Lab Assist narrative,Screening - Last Screening Date,Verification of Eligibility - Verification Date,Base-line Oral and Conditions Questions - Are you limited in what you can eat?,Base-line Oral and Conditions Questions - Dental Clinic,Base-line Oral and Conditions Questions - Did the client commit to referral?,Base-line Oral and Conditions Questions - Did you help in these other areas?,Base-line Oral and Conditions Questions - Do you need help in these other areas?,Base-line Oral and Conditions Questions - Have you ever felt that the appearance of your mouth and teeth affected your quality of life?,Base-line Oral and Conditions Questions - How would you describe the condition of your mouth and teeth?,Base-line Oral and Conditions Questions - What is the next step for client encounter?,Base-line Oral and Conditions Questions - What is the next step for clients encounter?,Case Management Information - Dental Clinic,Case Management Information - Transportation Barriers,Case Management Information - What barriers have kept you from finding a consistent dental home? (Select all that apply),Case Management Information - What is the outcome for the client?,Clinic Information - Action: Upload Before Picture on Finish Tab,Clinic Information - Is patient of record date if yes last seen?,Clinic Information - Was initial appointment made?,Clinic Information - Was initial transportation assistance or referral provided?,Demographic Information - Gender,Demographic Information - Household Income Level,Demographic Information - How did you first hear about the program?,Demographic Information - Preferred Language/Language Needed,Demographic Information - Race/Ethnicity,Demographic Information - What other assistance programs are you on?,Demographic Information - When is the last time you visited your dentist?,oec_What care was provided to the enrollee? (Select all that apply),Presumptive Eligibility - Dental Insurance?,Presumptive Eligibility - Do they pre-qualify for SMILE ON 60+?,Presumptive Eligibility - What is the next step for clients encounter?,Presumptive Eligibility - What is the next step for clients encounter?,Screening - # of Lower Natural Teeth,Screening - # of Upper Natural Teeth,Screening - Do you have removable dentures?,Screening - Do you usually wear your denture(s) when you eat?,Screening - Experienced any problems related to mouth/teeth during past 12 months?,Screening - How often do you brush your teeth?,Screening - Need for Periodontal Care,Screening - Root Fragments,Screening - Suspicious Soft Tissue Lesions,Screening - Treatment Urgency,Screening - Untreated Decay,Site Information - Enroller ID,Site Information - Enroller Location,Treatment Plan - Action: Upload After Picture on Finish Tab,Treatment Plan - Is the treatment plan completed?,Treatment Plan - Was a treatment plan developed?,"Treatment Plan - Was the enrollee able to be restored to function (can chew) and ""social six"" esthetics (top front six teeth are present and disease free)",Verification of Eligibility - Transportation Arranged,Verification of Eligibility - Verification of Eligibility
228,92968098,SMILE ON 60+ Clinical,9/27/202

In [37]:
print(clean_df['Treatment Plan - Is the treatment plan completed?'].value_counts())
print(clean_df['Treatment Plan - Was the enrollee able to be restored to function (can chew) and "social six" esthetics (top front six teeth are present and disease free)'].value_counts())

No     19382
Yes     7823
Name: Treatment Plan - Is the treatment plan completed?, dtype: int64
No     12260
Yes     9794
Name: Treatment Plan - Was the enrollee able to be restored to function (can chew) and "social six" esthetics (top front six teeth are present and disease free), dtype: int64


In [34]:
outcomes = clean_df[['CallerNum',
                     'Treatment Plan - Is the treatment plan completed?',
                     'Treatment Plan - Was the enrollee able to be restored to function (can chew) and "social six" esthetics (top front six teeth are present and disease free)',
                     'Birth_Year',
                      'Presumptive Eligibility - Do they pre-qualify for SMILE ON 60+?',
                      'Base-line Oral and Conditions Questions - Are you limited in what you can eat?',
                        'Treatment Plan - Was a treatment plan developed?',
                        ]]
outcomes.head()

,CallerNum,Treatment Plan - Is the treatment plan completed?,"Treatment Plan - Was the enrollee able to be restored to function (can chew) and ""social six"" esthetics (top front six teeth are present and disease free)",Birth_Year,Presumptive Eligibility - Do they pre-qualify for SMILE ON 60+?,Base-line Oral and Conditions Questions - Are you limited in what you can eat?,Treatment Plan - Was a treatment plan developed?
0,3638169,NaN,NaN,1960,Yes,Yes,NaN
1,4431194,NaN,NaN,1959,Yes,Yes,NaN
2,2941341,No,No,1942,Yes,No,Yes
3,3823933,No,No,1960,Yes,Yes,Yes
4,3348256,No,No,1955,Yes,Yes,Yes


In [55]:
print(outcomes.nunique())

CallerNum                                                                                                                                                     10711
Treatment Plan - Is the treatment plan completed?                                                                                                                 2
Treatment Plan - Was the enrollee able to be restored to function (can chew) and "social six" esthetics (top front six teeth are present and disease free)        2
Birth_Year                                                                                                                                                      124
Presumptive Eligibility - Do they pre-qualify for SMILE ON 60+?                                                                                                   2
Base-line Oral and Conditions Questions - Are you limited in what you can eat?                                                                                    2
Treatment Plan -

In [56]:
print(outcomes['CallerNum'].value_counts())

-1          63
 3212379    54
 2441776    43
 2758373    42
 2431132    37
            ..
 3604783     1
 2982163     1
 3852376     1
 2826507     1
 2451720     1
Name: CallerNum, Length: 10711, dtype: int64


In [65]:
compression_opts = dict(method='zip', archive_name='out.csv')
outcomes.to_csv('out.zip', index=False, compression=compression_opts)

In [52]:
completed_both = outcomes[(outcomes['Treatment Plan - Is the treatment plan completed?'] == "Yes") & (outcomes['Treatment Plan - Was the enrollee able to be restored to function (can chew) and "social six" esthetics (top front six teeth are present and disease free)'] == 'Yes')]
completed_both.head()

,CallerNum,Treatment Plan - Is the treatment plan completed?,"Treatment Plan - Was the enrollee able to be restored to function (can chew) and ""social six"" esthetics (top front six teeth are present and disease free)",Birth_Year,Presumptive Eligibility - Do they pre-qualify for SMILE ON 60+?,Base-line Oral and Conditions Questions - Are you limited in what you can eat?,Treatment Plan - Was a treatment plan developed?
9,2887352,Yes,Yes,1959,Yes,Yes,Yes
21,2750607,Yes,Yes,1953,Yes,No,Yes
24,3756941,Yes,Yes,1960,Yes,No,Yes
26,2957321,Yes,Yes,1943,Yes,NaN,Yes
32,3964791,Yes,Yes,1956,Yes,Yes,Yes


In [54]:
print(completed_both.nunique())

CallerNum                                                                                                                                                     3731
Treatment Plan - Is the treatment plan completed?                                                                                                                1
Treatment Plan - Was the enrollee able to be restored to function (can chew) and "social six" esthetics (top front six teeth are present and disease free)       1
Birth_Year                                                                                                                                                      79
Presumptive Eligibility - Do they pre-qualify for SMILE ON 60+?                                                                                                  2
Base-line Oral and Conditions Questions - Are you limited in what you can eat?                                                                                   2
Treatment Plan - Was a

In [45]:
print(completed_both['Treatment Plan - Is the treatment plan completed?'].value_counts())

Yes    6755
Name: Treatment Plan - Is the treatment plan completed?, dtype: int64


In [66]:
d = dtale.show(co)

#open it in a new window in browser

d.open_browser()